# Add Waveforms
Here we combine the experiment-specific stationXML file created in GLNN_StationXML.ipynb with the recorded AE waveforms in a tpc5 file to initialize the experiment ASDF file.

ElSys has provided a module tpc5.py __[available on their website](https://www.elsys-instruments.com/en/support/tpc5_fileformat.php)__ for easy interpretation.

In [6]:
import tpc5 # get from ElSys, see link above
import h5py
import numpy as np
import pyasdf
from obspy import Stream, Trace
from obspy.core import Stats

First set up the ASDF dataset to fill

In [4]:
ds = pyasdf.ASDFDataSet('020918_shear/020918ASDF.h5', compression='gzip-3')
ds.add_stationxml('020918_shear/GLNNstations_020918.xml')

An experiment might be made up of multiple files. Display the tpc5 files present in the experiment folder:

In [3]:
!ls 020918_shear/*.tpc5

020918_shear/bd1.tpc5  020918_shear/run.tpc5
020918_shear/bd2.tpc5  020918_shear/timing.tpc5


In [2]:
f = h5py.File("020918_shear/bd1.tpc5", "r")

In [5]:
# we need to know how many blocks to read
# all channels have the same number of blocks, use channel 1
cg = f[tpc5.getChannelGroupName(1)]
len(cg['blocks'].keys())

1

In [8]:
# read the raw data from each channel into a stream, one trace at a time
# first build some basic Stats
stats = Stats()
stats.network = 'L0'
stats.channel = 'FHZ'

# get trace stats


In [14]:
ds.waveforms.L0_AE09.StationXML.networks[0].stations[0]
# ASDF appears to dump the extra attribute of the StationXML
# Add the (x,y) locations as auxiliary data instead

Station AE09 (BP3)
	Station Code: AE09
	Channel Count: 1/None (Selected/Total)
	None - 
	Access: None 
	Latitude: 37.87, Longitude: -122.26, Elevation: 100.0 m
	Available Channels:
		AE09.00.FHZ

## Extra notes
Accessing tpc5 and hdf5 files can be a bit confusing. Here are some reminders:

In [ ]:
# to view the keys of an hdf5 file:
list(f.keys())

# to then access one of the keys:
f['key']